In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [2]:
city = 'newberg'

In [11]:
#create a dic where the key is the city name and the value is the bounding box
city_bound = {'newberg':[-122.98861694,45.28316879,-122.92491913,45.33226395],'chicago':[-88.15451813,41.55007553,-87.58644104,42.09182739]}

In [3]:
df = pd.read_csv('../../../data/shared/labels-with-tags-chu.csv')

In [4]:
#select city == current city
df = df[df['city'] == city]
df

,city,label_id,label_type,severity,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,global_attribute_id,tag_list,description,lat,lng
368554,newberg,8974,NoSidewalk,3.0,1,83.5000,-25.062500,268.615570,1.862495,bf66138c-6ecf-4bea-8b0a-7485b5b171e9,7297663.0,street has a sidewalk,NaN,45.293749,-122.978248
368555,newberg,9153,NoSidewalk,4.0,1,256.1250,-29.875000,268.797607,1.250404,bf66138c-6ecf-4bea-8b0a-7485b5b171e9,7297601.0,NaN,NaN,45.284772,-122.977772
368556,newberg,9154,NoSidewalk,4.0,1,256.1250,-29.875000,268.797607,1.250404,bf66138c-6ecf-4bea-8b0a-7485b5b171e9,7297601.0,NaN,NaN,45.284808,-122.977776
368557,newberg,9813,CurbRamp,1.0,1,34.6875,-5.330357,223.299789,-0.371170,25d1bc9a-56a4-433b-8ffb-82309fecf949,7304745.0,narrow,NaN,45.302033,-122.979015
368558,newberg,8969,NoSidewalk,4.0,1,94.9375,-19.437500,269.590149,2.145340,bf66138c-6ecf-4bea-8b0a-7485b5b171e9,7297626.0,NaN,NaN,45.293657,-122.979172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390028,newberg,18339,NoCurbRamp,NaN,1,132.8125,-35.000000,249.037506,0.641014,e76a46bc-4cab-4943-bbf8-92015a186bc6,7298330.0,no alternate route,NaN,45.298012,-122.979179
390029,newberg,18340,NoCurbRamp,NaN,1,132.8125,-35.000000,249.037506,0.641014,e76a46bc-4cab-4943-bbf8-92015a186bc6,7298356.0,no alternate route,NaN,45.298038,-122.979156
390030,newberg,18340,NoCurbRamp,NaN,1,132.8125,-35.000000,249.037506,0.641014,e76a46bc-4cab-4943-bbf8-92015a186bc6,7297863.0,no alternate route,NaN,45.298038,-122.979156
390031,newberg,18341,NoCurbRamp,NaN,1,76.5625,-27.687500,249.037506,0.641014,e76a46bc-4cab-4943-bbf8-92015a186bc6,7298358.0,no alternate route,NaN,45.298103,-122.979141


In [6]:
#get Nan counts of global attribute id
df['global_attribute_id'].isna().sum()

1201

In [7]:
#turn df into geodataframe
geometry = [Point(xy) for xy in zip(df.lng, df.lat)]
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

In [20]:
#fix the geometry to the city bound
o_input = gdf.cx[city_bound[city][0]:city_bound[city][2], city_bound[city][1]:city_bound[city][3]]

In [21]:
#o_input.explore()

In [22]:
#select only label id taglist and description 
o_input = o_input[['label_id', 'tag_list', 'description']]

In [23]:
#select tag list that are not nan
tag = o_input.drop(columns=['description'])
tag = tag[tag['tag_list'].notna()]
# create a new column with the length of the tag list 
tag['tag_count'] = tag['tag_list'].str.split(',').str.len()
#drop tag_list column
tag = tag.drop(columns=['tag_list'])

In [25]:
#merge with o_input
o_input = o_input.merge(tag, on='label_id', how='left')

In [27]:
# #turn nan to 0 non nan to 1
o_input['tag_list'] = o_input['tag_list'].notna().astype('int')
o_input['description'] = o_input['description'].notna().astype('int')
#fill nan with 0
o_input = o_input.fillna(0)

In [29]:
#group by label id and keep max
o_input= o_input.groupby('label_id').max()
#reset index
o_input = o_input.reset_index()

In [31]:
o_input

,label_id,tag_list,description,tag_count
0,8,1,0,1.0
1,9,1,0,1.0
2,10,0,1,0.0
3,11,0,1,0.0
4,12,0,0,0.0
...,...,...,...,...
16841,18337,1,0,2.0
16842,18338,1,0,1.0
16843,18339,1,0,1.0
16844,18340,1,0,1.0


In [33]:
#read in the raw labels
labels = pd.read_csv('../../../data/{}/processed-labels/labels_raw_{}.csv'.format(city, city))

In [34]:
#create a new df with only label id from labels
label_ids = labels[['label_id']]
#merge label ids with o_input
o_input = pd.merge(label_ids, o_input, on='label_id', how='left')
#fill nan with 0
o_input = o_input.fillna(0).sort_values(by='label_id').reset_index(drop=True)
o_input

,label_id,tag_list,description,tag_count
0,8,1.0,0.0,1.0
1,9,1.0,0.0,1.0
2,10,0.0,1.0,0.0
3,11,0.0,1.0,0.0
4,12,0.0,0.0,0.0
...,...,...,...,...
16842,18337,1.0,0.0,2.0
16843,18338,1.0,0.0,1.0
16844,18339,1.0,0.0,1.0
16845,18340,1.0,0.0,1.0


In [35]:
#rename columns tag_list to tags
o_input.rename(columns={'tag_list':'tag'}, inplace=True)
#change the order of columns to label_id, tags, tags_count, description
o_input= o_input[['label_id', 'tag', 'tag_count', 'description']]

In [37]:
#export to csv 
o_input.to_csv('../../../data/{}/features/optional_input.csv'.format(city), index=False)